In [111]:
%matplotlib notebook
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import contextily as cx
import xml.etree.ElementTree as ET
from shapely import wkt
from IPython.display import Image
import random


In [6]:
# trip distribution by numbers
# merged_data.groupby(by=['COUNTY_NM', 'dest_TAZ'])['demand'].agg('sum')

COUNTY_NM  dest_TAZ 
King       King         5040054
           Kitsap          4615
           Pierce        190763
           Snohomish     269506
Kitsap     King            4439
           Kitsap        583266
           Pierce         26688
           Snohomish        487
Pierce     King          190369
           Kitsap         26752
           Pierce       1711413
           Snohomish       1040
Snohomish  King          269150
           Kitsap           496
           Pierce          1072
           Snohomish    1590653
Name: demand, dtype: int64

In [7]:
# trip distribution by percentages
# merged_data.groupby(by=['COUNTY_NM', 'dest_TAZ'])['demand'].agg('sum')/merged_data.demand.sum() *100

COUNTY_NM  dest_TAZ 
King       King         50.854349
           Kitsap        0.046566
           Pierce        1.924806
           Snohomish     2.719326
Kitsap     King          0.044790
           Kitsap        5.885178
           Pierce        0.269283
           Snohomish     0.004914
Pierce     King          1.920831
           Kitsap        0.269929
           Pierce       17.268226
           Snohomish     0.010494
Snohomish  King          2.715734
           Kitsap        0.005005
           Pierce        0.010817
           Snohomish    16.049753
Name: demand, dtype: float64

1- kitsap-kitsap/pierce trips are not relevant to king county, and percent of trips between king/snohomish and kitsap is 0.046566+ 0.044790+0.004914+0.005005 = 0.01 % 
- therefore we fully eliminate trips to/from kitsap 

2- snohomish-pierce and pierce-snohomish that pass trough king county are negligible (0.010817+0.010494 = 0.02%)
- we eliminate trips between Snohomish and Pierce county

3- trips within pierce and snohomish are not relevant to king county
- we eliminate trips within Snohomish and Pierce county

### therefore, we only keep trips 1- within king county, 2- between king and pierce, and 3- between king and Snohomish



## For Pierce and Snohomish, 
it seems that majority of trips take place either between pierce and south of downtown OR snohomish and north of downtown, so what we do is keeping only the trips between snohomish/pierce and sumo TAZs

### final stats before further filterations



In [101]:
# filtered_data.groupby(by=['COUNTY_NM', 'dest_TAZ'])['demand'].agg('sum')

COUNTY_NM  dest_TAZ 
King       King         5040054
           Pierce        190763
           Snohomish     269506
Pierce     King            7352
Snohomish  King           18304
Name: demand, dtype: int64

In [10]:
# filtered_data.groupby(by=['COUNTY_NM', 'dest_TAZ'])['demand'].agg('sum')/filtered_data.demand.sum() *100

COUNTY_NM  dest_TAZ 
King       King         84.566906
           Pierce        3.200806
           Snohomish     4.522033
Pierce     King          3.194195
Snohomish  King          4.516059
Name: demand, dtype: float64

### Selecting TAZs and working with them in this notebook:
we run the code below in qgis python plugin:

<pre>
dir = '/Users/stlp/Library/CloudStorage/GoogleDrive-soheil99@uw.edu/My Drive/reasearch codes/SUMO/MatSumo-main/SUMO/demand generation/regions'
os.chdir(dir)
def get_selected_TAZs(address):
    layer = QgsProject.instance().mapLayersByName("taz2010")[0]
    selected_features = layer.selectedFeatures()
    attribute_list = [feature['TAZ'] for feature in selected_features]
    with open(address, 'w') as f:
        f.write(str(attribute_list))
        f.close()
</pre>
we select some regions in taz2010.shp and then use the command above like: `get_selected_TAZ('/1.txt')`
and read selected regions in qgis here using the function below:

In [112]:
def _read_region(address):
    with open(address, 'r') as f:
        content = f.read()
        data_list = eval(content)
    return data_list

def read_regions(address):
    if type(address)==list:
        outputs = []
        for adrs in address:
            outputs.append(_read_region(adrs))
        return outputs
    else:
        return _read_region(address)
            
#     with open(address, 'r') as f:
#         content = f.read()
#         data_list = eval(content)
#     return data_list

# Conversion algorithm
So far, I was trying to eliminate useless trips. There are some trips that we ARE going to use: trips with sumo_TAZs in one end (without over-3700 TAZs or kitsap TAZs)

I think the best way to deal with these trips is by starting with trips involving sumo_TAZs and then gradually add different scenarios in which vehicles will pass through sumo_TAZs.


In [113]:
# Load shapefiles
taz_data = gpd.read_file('taz2010.shp')

# Load CSV file
trip_data = pd.read_csv("psrc_vehicle_trips.csv")

# load sumo tazs
tree = ET.parse('Taz_with_pseudo_link.add.xml')
root = tree.getroot()
sumo_taz = []
for taz in root.findall('taz'):
    tazid = taz.get('id')
    if tazid != 'taz_13':
        sumo_taz.append(int(tazid))

In [114]:
# we obtain list of TAZs for each of counties so that we can use them during filtering single trips dataframe
king = list(taz_data[taz_data['COUNTY_NM']=='King'].TAZ.values)
pierce = list(taz_data[taz_data['COUNTY_NM']=='Pierce'].TAZ.values)
kitsap = list(taz_data[taz_data['COUNTY_NM']=='Kitsap'].TAZ.values)
snohomish = list(taz_data[taz_data['COUNTY_NM']=='Snohomish'].TAZ.values)

In [115]:
len(trip_data)

9985425

Eliminate over-3700s, Kitsap, and port trips

In [116]:
trip_data = trip_data[~(trip_data.otaz.isin(kitsap)|trip_data.dtaz.isin(kitsap))]
trip_data = trip_data[~((trip_data.otaz>3700)|(trip_data.dtaz>3700))]
ports = [444, 445, 449, 478]
trip_data = trip_data[~(trip_data.otaz.isin(ports)|trip_data.dtaz.isin(ports))]
len(trip_data)

9250192

## TAZ conversion for sumo_TAZs included trips

In [117]:
trip_data['original_o'] = trip_data.otaz
trip_data['original_d'] = trip_data.dtaz
trip_data['original_deptm'] = trip_data.deptm
# adding more columns will increase computation time in next steps

In [214]:
converted_data = trip_data[trip_data.otaz.isin(sumo_taz) | trip_data.dtaz.isin(sumo_taz)]
len(converted_data)

608908

In [215]:
duplicate_rows = trip_data[trip_data.duplicated(subset=['otaz', 'dtaz', 'deptm'], keep=False)]
print(duplicate_rows)

         otaz  dtaz  deptm  travtime  original_o  original_d  original_deptm
126         1     5    482         5           1           5             482
127         1     5    482         6           1           5             482
1244        2     1   1107         2           2           1            1107
1245        2     1   1107         5           2           1            1107
1290        2     2   1018         4           2           2            1018
...       ...   ...    ...       ...         ...         ...             ...
9331718  3466  3466   1013         3        3466        3466            1013
9331724  3466  3466   1040        22        3466        3466            1040
9331725  3466  3466   1040         1        3466        3466            1040
9331739  3466  3466   1123        52        3466        3466            1123
9331740  3466  3466   1123        16        3466        3466            1123

[567030 rows x 7 columns]


### for Pierce and Snohomish trips
Based on map of highways, we can assign pierce to I5-south (5002, 5003) and snohomish to I5-north (5000, 5001).  It is not exact and there could be many many trips passing through bellevue instead of I-5.

In [216]:
# interval = 15
def get_intra_TAZ_traveltime(o_list, d_list, interval=15, exact_time = None):
    if exact_time is None:
        trips = trip_data[trip_data.otaz.isin(o_list) & trip_data.dtaz.isin(d_list)]
        average = round(trips.travtime.mean())
        time_offsets = list()
        for start in range(0,1440, interval):
            end = start+interval
            interval_trips = trips[trips.deptm.isin(range(start, end))]
            average_time = interval_trips.travtime.mean()
            if pd.isna(average_time):
                time_offsets.append(average)
            else:
                time_offsets.append(round(average_time))
        return time_offsets
    else:
        trips = trip_data[trip_data.otaz.isin(o_list) & 
                          trip_data.dtaz.isin(d_list) &
                          trip_data.deptm.isin(range(int(exact_time-interval/2), int(exact_time+interval/2)+1))]

        average = trips.travtime.mean()
        return average

In [217]:
def get_random_TAZ(taz_list):
    if type(taz_list)==list:
        return random.choice(taz_list)
    else:
        return taz_list

In [218]:
def convert1(invalid_TAZs, near_pseudo_TAZs, pseudo_taz_in, pseudo_taz_out, df=None):
    if df is None:
            df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    invalid_to_near_tt = get_intra_TAZ_traveltime(invalid_TAZs, near_pseudo_TAZs)
    print(f"it'll take {len(df)} iterations.")
    for index, row in tqdm(df.iterrows()):
        interval = int(row.deptm//15)
        if row.otaz in invalid_TAZs:
            row.otaz = get_random_TAZ(pseudo_taz_in)
            row.deptm += invalid_to_near_tt[interval]
        elif row.dtaz in invalid_TAZs:
            row.dtaz = get_random_TAZ(pseudo_taz_out)
        converted_data.loc[index, :] = row 

In [219]:
sumo_s = read_regions('regions/sumo_south.txt')
sumo_n = [taz for taz in sumo_taz if taz not in sumo_s]
sumo_e = read_regions('regions/sumo_east.txt') 
sumo_w = [taz for taz in sumo_taz if taz not in sumo_e]
near_i90 = read_regions('regions/near_i90.txt')

In [220]:
Image(url="regions/i5.png", width=300, height=300)

In [221]:
near_i5n=[381, 382, 406, 407, 428, 430]
near_i5s=[649, 650, 657, 669, 670]    

In [222]:
convert1(snohomish, near_i5n, 5000, 5001)

it'll take 36268 iterations.


36268it [00:52, 690.20it/s]


In [223]:
convert1(pierce, near_i5s, 5002, 5003)

it'll take 14553 iterations.


14553it [00:27, 535.40it/s]


### for trips within king county

In [224]:
Image(url="regions/SR99N.png", width=300, height=300)

In [225]:
SR99n = read_regions('regions/SR99N.txt')
nearSR99n = read_regions('regions/near_SR99N.txt')

In [226]:
convert1(SR99n, nearSR99n, 5010, 5011)

it'll take 41114 iterations.


41114it [00:58, 703.59it/s]


 --------

In [227]:
Image(url="regions/i5n1.png", width=300, height=300)

In [228]:
i5n1 = read_regions('regions/i5n1.txt')

In [229]:
convert1(i5n1, near_i5n, 5000, 5001)

it'll take 45523 iterations.


45523it [01:09, 659.00it/s]


___

In [230]:
Image(url="regions/i5n2.png", width=300, height=300)

In [231]:
i5n2 = read_regions('regions/i5n2.txt')
convert1(i5n2, near_i5n, 5000, 5001)

it'll take 15673 iterations.


15673it [00:24, 633.39it/s]


--------

In [232]:
Image(url="regions/west_king1.png", width=600, height=300)

In [233]:
west_king1 = read_regions('regions/west_king1.txt')
df = converted_data[converted_data.otaz.isin(west_king1)|converted_data.dtaz.isin(west_king1)]

In [234]:
# if trips go to upper downtown:
df1 = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
convert1(west_king1, near_i5n, 5000, 5001, df=df1)

it'll take 8381 iterations.


8381it [00:12, 683.52it/s]


In [235]:
# if trips go to lower downtown:
df2 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
convert1(west_king1, near_i90, 5004, 5005, df=df2)

it'll take 4286 iterations.


4286it [00:05, 730.33it/s]


---------

In [236]:
Image(url="regions/west_king2.png", width=600, height=300)

In [237]:
west_king2 = read_regions('regions/west_king2.txt')
df = converted_data[converted_data.otaz.isin(west_king2)|converted_data.dtaz.isin(west_king2)]

In [238]:
# if trips go to upper downtown:
df1 = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
convert1(west_king2, near_i5n, 5000, 5001, df=df1)

it'll take 11223 iterations.


11223it [00:21, 526.04it/s]


In [239]:
# if trips go to lower downtown:
df2 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
convert1(west_king2, near_i90, 5004, 5005, df=df2)

it'll take 6167 iterations.


6167it [00:09, 648.46it/s]


-----

In [240]:
Image(url="regions/west_king3.png", width=500, height=300)

In [241]:
west_king3 = read_regions('regions/west_king3.txt')
convert1(west_king3, near_i90, 5004, 5005)

it'll take 33163 iterations.


33163it [00:55, 601.44it/s]


-----

In [242]:
Image(url="regions/west_king4.png", width=600, height=300)

In [243]:
west_king4 = read_regions('regions/west_king4.txt')
convert1(west_king4, near_i90, 5004, 5005)

it'll take 11397 iterations.


11397it [00:29, 392.87it/s]


-----

In [244]:
Image(url="regions/west_king5.png", width=600, height=300)

In [245]:
west_king5 = read_regions('regions/west_king5.txt')
convert1(west_king5, near_i5s, 5002, 5003)

it'll take 4920 iterations.


4920it [00:12, 399.94it/s]


-----

In [246]:
Image(url="regions/west_king6.png", width=300, height=300)

In [247]:
west_king6 = read_regions('regions/west_king6.txt')
convert1(west_king6, near_i5s, 5002, 5003)

it'll take 17509 iterations.


17509it [00:31, 556.43it/s]


--------

In [248]:
Image(url="regions/vashon.png", width=300, height=300)

In [249]:
vashon = read_regions('regions/vashon.txt')
nearsr99s = read_regions('regions/near_SR99S.txt')
convert1(vashon, nearsr99s, 5008, 5009)

it'll take 1206 iterations.


1206it [00:01, 692.35it/s]


------

In [250]:
Image(url="regions/southwest1.png", width=300, height=300)

In [251]:
sw1 = read_regions('regions/southwest1.txt')
df = converted_data[converted_data.otaz.isin(sw1)|converted_data.dtaz.isin(sw1)]

In [252]:
# if trips go to west downtown:
df1 = df[df.otaz.isin(sumo_w)|df.dtaz.isin(sumo_w)]
convert1(sw1, nearsr99s, 5008, 5009, df=df1)

it'll take 19529 iterations.


19529it [00:27, 704.01it/s]


In [253]:
# if trips go to east downtown:
df2 = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
convert1(sw1, near_i5s, 5002, 5003, df=df2)

it'll take 4512 iterations.


4512it [00:06, 685.53it/s]


In [254]:
Image(url="regions/southwest2.png", width=300, height=300)


In [255]:
sw2 = read_regions('regions/southwest2.txt')
df = converted_data[converted_data.otaz.isin(sw2)|converted_data.dtaz.isin(sw2)]

In [256]:
# if trips go to west downtown:
df1 = df[df.otaz.isin(sumo_w)|df.dtaz.isin(sumo_w)]
convert1(sw2, nearsr99s, 5008, 5009, df=df1)

it'll take 30346 iterations.


30346it [00:43, 697.14it/s]


In [257]:
# if trips go to east downtown:
df2 = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
convert1(sw2, near_i5s, 5002, 5003, df=df2)

it'll take 6100 iterations.


6100it [00:08, 683.41it/s]


----

In [258]:
Image(url="regions/i5s11.png", width=300, height=1000)

In [259]:
i5s1 = read_regions('regions/i5s1.txt')
convert1(i5s1, near_i5s, 5002, 5003)

it'll take 26281 iterations.


26281it [00:38, 679.35it/s]


In [260]:
Image(url="regions/i5s2.png", width=300, height=300)

In [261]:
i5s2 = read_regions('regions/i5s2.txt')
convert1(i5s2, near_i5s, 5002, 5003)

it'll take 14231 iterations.


14231it [00:25, 549.04it/s]


-----

In [262]:
Image(url="regions/southeast.png", width=400, height=300)

In [263]:
se = read_regions('regions/southeast.txt')

In [264]:
# if trips go to east downtown:
df = converted_data[converted_data.otaz.isin(se)|converted_data.dtaz.isin(se)]
df = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
convert1(se, [636], 636, 636, df=df)

it'll take 4056 iterations.


4056it [00:06, 586.58it/s]


In [265]:
# if trips go to south downtown:
df = converted_data[converted_data.otaz.isin(se)|converted_data.dtaz.isin(se)]
df = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
convert1(se, [650], 650, 650, df=df)

it'll take 5070 iterations.


5070it [00:06, 725.45it/s]


In [266]:
# if trips go to north downtown:
df = converted_data[converted_data.otaz.isin(se)|converted_data.dtaz.isin(se)]
df = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
convert1(se, near_i5s, 5002, 5003, df=df)

it'll take 6196 iterations.


6196it [00:08, 763.41it/s]


----

In [267]:
Image(url="regions/east1.png", width=400, height=300)

In [268]:
east1 = read_regions('regions/east1.txt')
boundary = read_regions('regions/east1_boundary.txt')
# convert1(east1, boundary, boundary, boundary)
convert1(east1, boundary, 7004, 7004)

it'll take 33580 iterations.


33580it [00:48, 690.29it/s]


----

In [269]:
Image(url="regions/east2.png", width=400, height=300)

In [270]:
east2 = read_regions('regions/east2.txt')
boundary = read_regions('regions/east2_boundary.txt')
# convert1(east2, boundary, boundary, boundary)
convert1(east2, boundary, 7005, 7005)

it'll take 18070 iterations.


18070it [00:27, 667.90it/s]


----

In [271]:
Image(url="regions/north1.png", width=400, height=300)

In [272]:
north1 = read_regions('regions/north1.txt')
boundary = read_regions('regions/north1_boundary.txt')
# convert1(north1, boundary, boundary, boundary)
convert1(north1, boundary, 7001, 7001)

it'll take 8357 iterations.


8357it [00:14, 596.53it/s]


----

In [273]:
Image(url="regions/north2.png", width=300, height=300)

In [274]:
north2 = read_regions('regions/north2.txt')
boundary = read_regions('regions/north2_boundary.txt')
# convert1(north2, boundary, boundary, boundary)
convert1(north2, boundary, 7002, 7002)

it'll take 16303 iterations.


16303it [00:24, 669.31it/s]


----

In [275]:
Image(url="regions/north_west.png", width=300, height=300)

In [276]:
NW = read_regions('regions/north_west.txt')
boundary = read_regions('regions/north3_boundary.txt')
df = converted_data[converted_data.otaz.isin(NW)|converted_data.dtaz.isin(NW)]

df1 = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
# convert1(NW, boundary, boundary, boundary, df=df1)
convert1(NW, boundary, 7003, 7003, df=df1)

df2 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
convert1(NW, nearSR99n, 5010, 5011, df=df2)

it'll take 21580 iterations.


21580it [00:39, 546.11it/s]


it'll take 3582 iterations.


3582it [00:06, 585.66it/s]


-------

In [277]:
Image(url="regions/6001.png", width=300, height=300)

In [278]:
to6001 = read_regions('regions/6001.txt')
near6001 = read_regions('regions/near6001.txt')

In [279]:
df = converted_data[converted_data.otaz.isin(to6001)|converted_data.dtaz.isin(to6001)]

df1 = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
convert1(to6001, near6001, 6001, 6001, df=df1)

df2 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
convert1(to6001, nearSR99n, 5010, 5011, df=df2)


it'll take 24296 iterations.


24296it [00:56, 427.17it/s]


it'll take 7463 iterations.


7463it [00:13, 535.56it/s]


----

In [280]:
Image(url="regions/south.png", width=300, height=300)

In [281]:
south = read_regions('regions/south.txt')
boundary = read_regions('regions/south_boundary.txt')
# convert1(south, boundary, boundary, boundary)
convert1(south, boundary, 7000, 7000)

it'll take 11355 iterations.


11355it [00:16, 704.25it/s]


-------

# TAZ conversion for trips passing through sumo_TAZs

For this stage, we assume some scenarios, find related trips in out main trip dataframe, process them, and then add them to our existing trips

In [282]:
def get_trips_in_between(src_df, taz_list1, taz_list2):
# make sure taz_list1 and taz_list2 have no TAZ within sumo_TAZs. 
# In that case, some of the trips that were already processed in converted_data will be included
    df = src_df[(src_df.otaz.isin(taz_list1) & src_df.dtaz.isin(taz_list2)) | 
                (src_df.otaz.isin(taz_list2) & src_df.dtaz.isin(taz_list1))]
    return df

In [283]:
def convert2(df, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2):
    zone1offsets = get_intra_TAZ_traveltime(zone1, near_new_z1)
    zone2offsets = get_intra_TAZ_traveltime(zone2, near_new_z2)
    new_z1_in, new_z1_out = new_z1[0], new_z1[1]
    new_z2_in, new_z2_out = new_z2[0], new_z2[1]
    
    print(f"it'll take {len(df)} iterations.")
    for index, row in tqdm(df.iterrows()):
        interval = int(row.deptm//15)
        
        if row.otaz in zone1 and row.dtaz in zone2:
            row.otaz = get_random_TAZ(new_z1_in)
            row.dtaz = get_random_TAZ(new_z2_out)
            row.deptm += zone1offsets[interval]
            
        elif row.otaz in zone2 and row.dtaz in zone1:
            row.otaz = get_random_TAZ(new_z2_in)
            row.dtaz = get_random_TAZ(new_z1_out)
            row.deptm += zone2offsets[interval]
#         converted_data.loc[index, :] = row 
        df.loc[index, :] = row

-----

In [284]:
Image(url="regions/scenarios.png", width=900, height=300)

In [285]:
#scenario 1:

# out_of_sumo_data = get_trips_in_between(trip_data, zone1, zone2) 
out_of_sumo_data = get_trips_in_between(trip_data, i5n2, i5s1) 
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
convert2(out_of_sumo_data, i5n2, [5000, 5001], near_i5n, i5s1, [5002, 5003], near_i5s)

it'll take 1406 iterations.


1406it [00:00, 4197.13it/s]


In [286]:
def make_pseudo_trips(zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2):
        df = get_trips_in_between(trip_data, zone1, zone2) 
        convert2(df, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
        return df

In [287]:
# scenario 2
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
df = make_pseudo_trips(i5n2, [5000, 5001], near_i5n, sw2, [5002, 5003], near_i5s)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])


it'll take 2462 iterations.


2462it [00:00, 4096.17it/s]


In [288]:
# scenario 3
east2_b = read_regions('regions/east2_boundary.txt')

# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(i5n2, [5000, 5001], near_i5n, east2, [east2_b, east2_b], east2_b)
df = make_pseudo_trips(i5n2, [5000, 5001], near_i5n, east2, [7005, 7005], east2_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 1301 iterations.


1301it [00:00, 4297.94it/s]


In [289]:
# scenario 4
east1_b = read_regions('regions/east1_boundary.txt')
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(i5n2, [5000, 5001], near_i5n, east1, [east1_b, east1_b], east1_b)
df = make_pseudo_trips(i5n2, [5000, 5001], near_i5n, east1, [7004, 7004], east1_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 2657 iterations.


2657it [00:00, 4183.19it/s]


In [290]:
# scenario 5
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(i5s2, [5002, 5003], near_i5s, east1, [east1_b, east1_b], east1_b)
df = make_pseudo_trips(i5s2, [5002, 5003], near_i5s, east1, [7004, 7004], east1_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 2196 iterations.


2196it [00:00, 4115.63it/s]


In [291]:
# scenario 6
north3_b = read_regions('regions/north3_boundary.txt')
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(NW, [north3_b, north3_b], north3_b, east1, [east1_b, east1_b], east1_b)
df = make_pseudo_trips(NW, [7003, 7003], north3_b, east1, [7004, 7004], east1_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 1588 iterations.


1588it [00:00, 4342.95it/s]


In [292]:
# scenario 7
north2_b = read_regions('regions/north2_boundary.txt')
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(north2, [north2_b, north2_b], north2_b, east2, [east2_b, east2_b], east2_b)
df = make_pseudo_trips(north2, [7002, 7002], north2_b, east2, [7005, 7005], east2_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 870 iterations.


870it [00:00, 4866.48it/s]


In [293]:
# scenario 8
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(NW, [north3_b, north3_b], north3_b, east2, [east2_b, east2_b], east2_b)
df = make_pseudo_trips(NW, [7003, 7003], north3_b, east2, [7005, 7005], east2_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 851 iterations.


851it [00:00, 4188.80it/s]


In [294]:
# scenario 9
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(i5s2, [5002, 5003], near_i5s, east2, [east2_b, east2_b], east2_b)
df = make_pseudo_trips(i5s2, [5002, 5003], near_i5s, east2, [7005, 7005], east2_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 1846 iterations.


1846it [00:00, 4291.40it/s]


In [295]:
# scenario 10
south_b = read_regions('regions/south_boundary.txt')
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(NW, [north3_b, north3_b], north3_b, south, [south_b, south_b], south_b)
df = make_pseudo_trips(NW, [7003, 7003], north3_b, south, [7000, 7000], south_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 1172 iterations.


1172it [00:00, 4218.30it/s]


In [296]:
# scenario 11
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
df = make_pseudo_trips(NW, [436, 436], north3_b, se, [[636, 650], [636, 650]], [636, 647, 648, 649, 650 ])
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 624 iterations.


624it [00:00, 4448.09it/s]


In [297]:
# scenario 12
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(NW, [north3_b, north3_b], north3_b, sw1, [5008, 5009], nearsr99s)
df = make_pseudo_trips(NW, [7003, 7003], north3_b, sw1, [5008, 5009], nearsr99s)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 2238 iterations.


2238it [00:00, 4449.74it/s]


In [298]:
# scenario 13
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
df = make_pseudo_trips((SR99n+to6001), [5010, 5011], nearSR99n,
                       sw1, [5008, 5009], nearsr99s)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 9355 iterations.


9355it [00:02, 3574.04it/s]


In [299]:
# scenario 14
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
df = make_pseudo_trips(i5n1, [5000, 5001], near_i5n,
                       i5s1, [5002, 5003], near_i5s)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 7321 iterations.


7321it [00:01, 3826.75it/s]


In [300]:
# scenario 15
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(i5n1, [5000, 5001], near_i5n, south, [south_b, south_b], south_b)
df = make_pseudo_trips(i5n1, [5000, 5001], near_i5n, 
                       south, [7000, 7000], south_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 4856 iterations.


4856it [00:01, 3591.17it/s]


In [301]:
# scenario 18
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
df = make_pseudo_trips(sw2, [5008, 5009], nearsr99s,
                       (SR99n+to6001), [5010, 5011], nearSR99n)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 14977 iterations.


14977it [00:04, 3247.68it/s]


In [302]:
# scenario 19
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(south, [south_b, south_b], south_b,
#                        (SR99n+to6001), [5010, 5011], nearSR99n)
df = make_pseudo_trips(south, [7000, 7000], south_b,
                       (SR99n+to6001), [5010, 5011], nearSR99n)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 7006 iterations.


7006it [00:02, 3277.63it/s]


In [303]:
# scenario 20
north1_b = read_regions('regions/north1_boundary.txt')
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(south, [south_b, south_b], south_b,
#                        north1, [5000, 5001], near_i5n)
df = make_pseudo_trips(south, [7000, 7000], south_b,
                       north1, [5000, 5001], near_i5n)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 388 iterations.


388it [00:00, 3798.85it/s]


In [304]:
out_of_sumo_data

,otaz,dtaz,deptm,travtime,original_o,original_d,original_deptm
24739,5000,5003,490,28,8,708,464
24740,5000,5003,506,30,8,709,478
24741,5000,5003,529,30,8,709,503
24742,5000,5003,887,24,8,711,869
24745,5000,5003,1001,36,8,718,977
...,...,...,...,...,...,...,...
1539539,7000,5001,626,18,726,408,618
1540589,7000,5001,1118,22,727,365,1109
1540590,7000,5001,910,22,727,367,902
1541385,7000,5001,991,24,728,365,983


In [305]:
len(converted_data)

608908

In [306]:
converted_data = pd.concat([converted_data, out_of_sumo_data])

In [307]:
len(converted_data)

672022

# making CSV file

In [ ]:
sort?

In [308]:
converted_data.to_csv('new_psrc_vehicle_trips_converted_taz.csv', index=False)

In [309]:
df = pd.read_csv('new_psrc_vehicle_trips_converted_taz.csv')

In [310]:
df


,otaz,dtaz,deptm,travtime,original_o,original_d,original_deptm
0,5010,428,517,27,1,428,498
1,5010,432,321,20,1,432,304
2,5010,432,375,24,1,432,360
3,5010,432,520,26,1,432,501
4,5010,432,527,26,1,432,510
...,...,...,...,...,...,...,...
672017,7000,5001,626,18,726,408,618
672018,7000,5001,1118,22,727,365,1109
672019,7000,5001,910,22,727,367,902
672020,7000,5001,991,24,728,365,983
